In [29]:
# Define the environments and packages
env_packages = {
    'e1': ['p1', 'p2'],
    'e2': ['p1'],
    'e3': ['p3', 'p4'],
    'e4': ['p4'],
    'e5': ['p5']
}


In [30]:
from collections import defaultdict
import itertools

# Function to calculate the similarity between two sets of packages
def package_similarity(packages1, packages2):
    set1 = set(packages1)
    set2 = set(packages2)
    return len(set1 & set2) / len(set1 | set2)

# Function to create the union of packages for given environments
def union_packages(env_list, all_packages):
    union_set = set()
    for env in env_list:
        union_set.update(all_packages[env])
    return list(union_set)

# Function to build the tree iteratively
def build_tree(env_packages):
    nodes = {env: {'packages': env_packages[env], 'children': {}} for env in env_packages}
    all_packages = env_packages.copy()
    
    while len(nodes) > 1:
        # Find the pair of environments with the highest similarity
        most_similar_pair = None
        highest_similarity = -1
        for (env1, node1), (env2, node2) in itertools.combinations(nodes.items(), 2):
            similarity = package_similarity(node1['packages'], node2['packages'])
            if similarity > highest_similarity:
                highest_similarity = similarity
                most_similar_pair = (env1, env2)
        
        env1, env2 = most_similar_pair
        new_env = f'{env1}_{env2}'
        new_packages = union_packages([env1, env2], all_packages)
        
        # Create the new merged node
        nodes[new_env] = {
            'packages': new_packages,
            'children': {
                env1: nodes.pop(env1),
                env2: nodes.pop(env2)
            }
        }
        # Update the all_packages dictionary with the new merged environment
        all_packages[new_env] = new_packages
    
    # The last remaining node is the root
    return nodes.popitem()[1]

# Build the tree starting from the root
tree = build_tree(env_packages)


In [32]:
# Function to print the tree with package counts in a readable format
def print_tree(node, indent=0):
    indent_str = '  ' * indent
    package_count = len(node['packages'])  # Calculate the number of packages
    
    print(f"{indent_str}Packages ({package_count}): {node['packages']}")
    
    if node['children']:
        for child_key, child in node['children'].items():
            print(f"{indent_str}Child: {child_key}")
            print_tree(child, indent + 1)

# Print the tree
print_tree(tree)



Packages (5): ['p1', 'p3', 'p4', 'p5', 'p2']
Child: e3_e4
  Packages (2): ['p3', 'p4']
  Child: e3
    Packages (2): ['p3', 'p4']
  Child: e4
    Packages (1): ['p4']
Child: e5_e1_e2
  Packages (3): ['p1', 'p2', 'p5']
  Child: e5
    Packages (1): ['p5']
  Child: e1_e2
    Packages (2): ['p1', 'p2']
    Child: e1
      Packages (2): ['p1', 'p2']
    Child: e2
      Packages (1): ['p1']
